In [1]:
import os
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from uuid import uuid4
uuid = uuid4()
PICKLENAME = f"qm_calcs_{uuid}.pickle"


from pathlib import Path
import numpy as np
import ase
from ase.io import read
from ase.visualize import view

import itertools
import sys

# sys.path.append("/pchem-data/meuwly/boittier/home/pyscfgpu/tests/supermolecule")
# from supermol import compute_interaction_energy

# sys.path.append("/pchem-data/meuwly/boittier/home/pyscfgpu/helper_functions/")
# from basis import *


import os
os.environ["CHARMM_HOME"] = "/pchem-data/meuwly/boittier/home/charmm"
os.environ["CHARMM_LIB_DIR"] = "/pchem-data/meuwly/boittier/home/charmm/build/cmake"

import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake

from pycharmm.lib import charmm as libcharmm


sys.path.append("/pchem-data/meuwly/boittier/home/project-mmml/eric-copy/")


from pycharmmCommands import header, pbcs, nbonds, heat, equi, dyna, mini

from tqdm import tqdm



packmol_input = str(Path("packmol.inp").absolute())

import os
import shutil
from io import BytesIO
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

import numpy as np

import ase
from ase import Atoms, io
from ase.data import covalent_radii
from ase.io.pov import get_bondpairs, set_high_bondorder_pairs
from ase.visualize.plot import plot_atoms


from sklearn.decomposition import PCA

default_color_dict = {
    "Cl": [102, 227, 115],
    "C": [61, 61, 64],
    "O": [240, 10, 10],
    "N": [10, 10, 240],
    "F": [0, 232, 0],
    "H": [232, 206, 202],
    "K": [128, 50, 100],
    "X": [200, 200, 200],
}



def render_povray(
    atoms, pov_name, rotation="0x, 0y, 0z", radius_scale=0.25, color_dict=None,
    vectors = None, angles = None, planes = None,
):

    # align the molecule to the principal axes
    pca = PCA(n_components=3)
    pca.fit(atoms.get_positions())
    atoms.set_positions(pca.transform(atoms.get_positions()))

    if color_dict is None:
        color_dict = default_color_dict

    path = Path(pov_name)
    pov_name = path.name
    base = path.parent

    radius_list = []
    for atomic_number in atoms.get_atomic_numbers():
        radius_list.append(radius_scale * covalent_radii[atomic_number])

    colors = np.array([color_dict[atom.symbol] for atom in atoms]) / 255

    bondpairs = get_bondpairs(atoms, radius=0.9)
    good_bonds = []
    good_bond_keys = []
    for _ in bondpairs:
        #  remove the Cl-Cl bonds
        if not (atoms[_[0]].symbol == "Cl" and atoms[_[1]].symbol == "Cl"):
            good_bonds.append(_)
            good_bond_keys.append((_[0], _[1]))
            good_bond_keys.append((_[1], _[0]))

    # create hydrogen bonds
    _pos = atoms.get_positions()
    _z = atoms.get_atomic_numbers()
    idx_onh = (_z == 8) | (_z == 1) | (_z == 7)
    idxs = np.where(idx_onh)[0]

    # create a mapping between atom idxs in the first atoms object
    # and the idxs in the new atoms object
    map = {}
    for i, idx in enumerate(idxs):
        map[i] = idx
    # create a new atoms object with only N, O and H atoms
    atoms_onh = Atoms(_z[idxs], _pos[idxs])
    bondpairs_onh = get_bondpairs(atoms_onh, radius=1.5)
    for _ in bondpairs_onh:
        if (map[_[0]], map[_[1]]) not in good_bond_keys:
            distance = np.linalg.norm(_pos[_[0]] - _pos[_[1]])
            # check that atom1 is H or N/O and atom2 is N/O or H
            if (_z[_[0]] == 1 and (_z[_[1]] == 7 or _z[_[1]] == 8)) or (
                _z[_[1]] == 1 and (_z[_[0]] == 7 or _z[_[0]] == 8)
            ):
                if 1.0 < distance < 3.5:
                    print(f"Adding bond between", map[_[0]], map[_[1]])
                    good_bonds.append(_)

    good_bonds = set_high_bondorder_pairs(good_bonds)

    kwargs = {  # For povray files only
        "transparent": True,  # Transparent background
        "canvas_width": 1028*2,  # Width of canvas in pixels
        "canvas_height": None,  # None,  # Height of canvas in pixels
        "camera_dist": 10,  # Distance from camera to front atom,
        "camera_type": "orthographic angle 0",  # 'perspective angle 20'
        "depth_cueing": False,
        "colors": colors,
        "bondatoms": good_bonds,
        "textures": ["jmol"] * len(atoms),
        'image_plane'  : None,
    }

    generic_projection_settings = {
        "rotation": rotation,
        "radii": radius_list,
    }

    povobj = io.write(
        pov_name, atoms, **generic_projection_settings, povray_settings=kwargs
    )


    from ase.io.utils import PlottingVariables
    pvars = PlottingVariables(ase_atoms, scale=1,
                              **generic_projection_settings)
    R = pvars.positions
    w = pvars.w
    h = pvars.h
    z0 = R[:, 2].max()
    shift = (w /2 , h /2 , z0)
    
    R = pvars.positions[:len(atoms)] - shift
    

    
    def center_normal(plane):
        plane = np.array(plane)
        center = plane.T.mean(axis=1)
        return center, np.linalg.norm(plane.T, axis=1)


    with open("render_povray.pov") as f:
        lines = f.readlines()
        
    with open(pov_name, "a") as f:
        for l in lines:
            f.write(l)
        
        def format_vec(array):
            """Povray array syntax"""
            return '<' + ', '.join(f"{x:>6.2f}" for x in tuple(array)) + '>'
                    
        if angles:

            angles = [[R[_[0]], R[_[1]], R[_[2]]] for _ in angles]
            for a in angles:
                dist = np.sum((a[2] - a[0])**2)**0.5
                print(dist)
                a = [format_vec(_) for _ in a]
                f.write(f"\nAngleMarker({a[0]}, {a[1]}, {a[2]}, {dist})\n")
    
        if planes:
            planes = [[R[_[0]], R[_[1]], R[_[2]]] for _ in planes]
            for p in planes:
                p = center_normal(p)
                p = [format_vec(_) for _ in p]
                f.write(f"\nPlane({p[0]}, {p[1]}, 0.1, 0.01)\n")
    
        if vectors:
            vectors = [[R[_[0]], R[_[1] ]] for _ in vectors]

            for v in vectors:
                v = [format_vec(_) for _ in v]
                f.write(f"\nArrow({v[0]}, {v[1]}, 0.02, 0.01)\n")


    
    povobj.render(
        povray_executable="/pchem-data/meuwly/boittier/home/miniforge3/envs/jaxphyscharmm/bin/povray"
    )
    png_name = pov_name.replace(".pov", ".png")
    shutil.move(png_name, base / png_name)
    
    return png_name, povobj
import pandas as pd


def process_data(intE_final_results, n_atoms=10):
    intE_results = pd.DataFrame([intE_final_results["intE_results"]])

    standard_res = {k:v for k,v in intE_final_results.items() if k != "intE_results"}
    standard_res.pop("results_A_ghost")
    standard_res.pop("results_B_ghost")


    def scalar_cols(key, v, n=n_atoms):
        axs = ["x", "y", "z"]
        return {key + f"_{i}_{axs[j]}": v[i][j] if i < v.shape[0] else 0 for i in range(n) for j in range(3)}
    
    def scalar_cols_hess(key, v, n=n_atoms):
        axs = ["x", "y", "z"]
        return {key + f"_{i}_{j}": v[i][j] if (i < v.shape[0] and j < v.shape[1]) else 0 for i in range(n*3) for j in range(n*3)}
    
    standard_res_scaler = { k:  { kk: vv for kk,vv in v.items() if kk in ["energy"]} for k,v in standard_res.items()}

    standard_res_gradient = { k:  [scalar_cols(kk, vv, n_atoms) for kk, vv in v.items() if kk == "gradient"] for k,v in standard_res.items()}
    gradients = {k: pd.DataFrame(standard_res_gradient[k]).to_numpy().reshape(n_atoms, 3) for k in standard_res_gradient.keys()}
    
    standard_res_hessian = { k:  [scalar_cols_hess(kk, vv, n_atoms) for kk, vv in v.items() if kk == "hessian"] for k,v in standard_res.items()}
    hessian = {k: pd.DataFrame(standard_res_hessian[k]).to_numpy().reshape(n_atoms, 3, n_atoms, 3) for k in standard_res_hessian.keys()}
    
    thermo_results = []
    keys = list(standard_res.keys())
    for key in keys:
        old  = standard_res[key]["thermo"]
        new = {}
        for k, v in old.items():
            if type(v) is tuple:
                if k != "rot_const":
                    value, unit = tuple(v)
                    if unit == "":
                        unit = "unitless"
                    new[k+"_"+unit] = value
                else:
                    value, unit = tuple(v)
                    new[k+"_x_"+unit] = value[0]
                    new[k+"_y_"+unit] = value[0]
                    new[k+"_z_"+unit] = value[0]
        thermo_results.append(new)
    
    thermo_results_df = pd.DataFrame(thermo_results, index=keys)

    output =     {    "thermo_results_df": thermo_results_df,
        "intE_results": intE_results, 
        "gradients": gradients, 
        "hessians": hessian, 
        "results": standard_res,
        }
    
    import numpy as np
    assert np.all(output["thermo_results_df"].index == ['results_AB', 'results_A', 'results_B'])
    formation_thermo = pd.DataFrame(output["thermo_results_df"].iloc[0] - output["thermo_results_df"].iloc[1] - output["thermo_results_df"].iloc[2])
    delta_thermo = pd.DataFrame(output["thermo_results_df"].iloc[1] - output["thermo_results_df"].iloc[2])
    mean_thermo = pd.DataFrame( (output["thermo_results_df"].iloc[1] - output["thermo_results_df"].iloc[2])/2)
    delta_keys = pd.DataFrame(delta_thermo)
    
    final_output = {
        "formation_thermo": formation_thermo,
        "delta_thermo": delta_thermo ,
        "mean_thermo": mean_thermo ,
        "delta_keys": delta_keys,
    }
    final_output.update(output)
    return final_output

dpi = 300
import matplotlib as mpl
mpl.rcParams['figure.dpi']= dpi

In [2]:
"/pchem-data/meuwly/boittier/home/charmm/toppar/par_all36_cgenff.prm"
"/pchem-data/meuwly/boittier/home/charmm/toppar/top_all36_cgenff.rtf"

'/pchem-data/meuwly/boittier/home/charmm/toppar/top_all36_cgenff.rtf'

In [3]:
generate_script = """

! Load topology and parameters
READ PARAM CARD NAME  /pchem-data/meuwly/boittier/home/charmm/toppar/par_all36_cgenff.prm
!READ RTF CARD NAME /pchem-data/meuwly/boittier/home/charmm/toppar/top_all36_cgenff.rtf


"""

In [ ]:
pycharmm.lingo.charmm_script(generate_script)

In [2]:
from pyscf import gto, scf, dft, cc

mol = gto.M(
    atom = 'C 0 0 0; H 0 0 1.1199',
    basis = 'ccpvdz',
    spin = 1,
    verbose=0)

mf = scf.UHF(mol).run()

# HF-based
mycc = cc.UCCSD(mf, frozen=1)
mycc.kernel()
print('CCSD(HF) energy', mycc.e_tot)

# DFT-based
ks = dft.UKS(mol)
ks.xc = 'blyp'
ks.kernel()

# Overwrite energy and mo_coeff
mf.e_tot = mf.energy_tot(dm=ks.make_rdm1()) # perhaps we should add this to ccsd.py
mf.mo_coeff = ks.mo_coeff

mycc = cc.UCCSD(mf, frozen=1)
mycc.kernel()
print('CCSD(DFT) energy', mycc.e_tot)

CCSD(HF) energy -38.377924130886356
CCSD(DFT) energy -38.376963827651


In [1]:
from pyscf import gto, scf, dft, cc

mol = gto.M(
    atom = 'C 0 0 0; H 0 0 1.1199',
    basis = 'ccpvtz',
    spin = 1,
    verbose=0)

mf = scf.UHF(mol).run()

# HF-based
mycc = cc.UCCSD(mf, frozen=1)
mycc.kernel()
print('CCSD(HF) energy', mycc.e_tot)

# DFT-based
ks = dft.UKS(mol)
ks.xc = 'blyp'
ks.kernel()

# Overwrite energy and mo_coeff
mf.e_tot = mf.energy_tot(dm=ks.make_rdm1()) # perhaps we should add this to ccsd.py
mf.mo_coeff = ks.mo_coeff

mycc = cc.UCCSD(mf, frozen=1)
mycc.kernel()
print('CCSD(DFT) energy', mycc.e_tot)

CCSD(HF) energy -38.40709565861507
CCSD(DFT) energy -38.4056547029635


In [4]:
from pyscf import gto, scf, dft, cc

mol = gto.M(
    atom = 'C 0 0 0; H 0 0 1.1199',
    basis = 'ccpv5z',
    spin = 1,
    verbose=0)

mf = scf.UHF(mol).run()

# HF-based
mycc = cc.UCCSD(mf, frozen=1)
mycc.kernel()
print('CCSD(HF) energy', mycc.e_tot)

# DFT-based
ks = dft.UKS(mol)
ks.xc = 'blyp'
ks.kernel()

# Overwrite energy and mo_coeff
mf.e_tot = mf.energy_tot(dm=ks.make_rdm1()) # perhaps we should add this to ccsd.py
mf.mo_coeff = ks.mo_coeff

mycc = cc.UCCSD(mf, frozen=1)
mycc.kernel()
print('CCSD(DFT) energy', mycc.e_tot)

CCSD(HF) energy -38.41746501592862
CCSD(DFT) energy -38.41527141942608


In [5]:
# Write Molden file
with open('output.molden', 'w') as f:
    molden.header(mol, f)
    molden.orbital_coeff(mol, mf.mo_coeff, f)

NameError: name 'molden' is not defined